In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go

In [2]:
noise_data = pd.read_csv("../Data/hourly_noisedata_2022.csv", header = 0, sep=',')
print(noise_data.head())

   month  day  hour                       description      lamax       laeq
0      1    1     0      MP 03: Naamsestraat 62 Taste  60.322528  57.126833
1      1    1     0    MP 05: Calvariekapel KU Leuven  53.230972  49.987639
2      1    1     0  MP 06: Parkstraat 2 La Filosovia  53.666056  50.752000
3      1    1     0            MP 07: Naamsestraat 81  50.056861  47.440222
4      1    1     1      MP 03: Naamsestraat 62 Taste  53.033583  50.853806


In [3]:
# Should we take the mean over all locations first? Or make it such that you can select the locations?

# take the mean value across all locations
noise_per_hour = noise_data.drop(columns='description', axis=1)
noise_per_hour = noise_per_hour.groupby(['month', 'day', 'hour']).mean()
noise_per_hour = noise_per_hour.reset_index()
noise_per_hour.head()

,month,day,hour,lamax,laeq
0,1,1,0,54.319104,51.326674
1,1,1,1,54.152799,51.563278
2,1,1,2,50.295141,47.712392
3,1,1,3,47.531120,45.182907
4,1,1,4,45.521949,42.996168


In [4]:
# Create a new column 'wk_day' in the dataframe
noise_per_hour['wk_day'] = None

# Iterate through each row in the DataFrame
for index, row in noise_per_hour.iterrows():
    # Extract the hour, day, and month values from the row
    hour = row['hour']
    day = row['day']
    month = row['month']
    
    # Create a datetime object for the given hour, day, month, and assuming the year is 2022
    dt = datetime.datetime(2022, month, day, hour)
    
    # Get the abbreviated weekday name (e.g., Mon, Tue, Wed) and assign it to the 'wk_day' column
    noise_per_hour.at[index, 'wk_day'] = dt.strftime("%a")

print(noise_per_hour.head())

   month  day  hour      lamax       laeq wk_day
0      1    1     0  54.319104  51.326674    Sat
1      1    1     1  54.152799  51.563278    Sat
2      1    1     2  50.295141  47.712392    Sat
3      1    1     3  47.531120  45.182907    Sat
4      1    1     4  45.521949  42.996168    Sat


In [5]:
weekday_order = ['Sun', 'Sat', 'Fri', 'Thu', 'Wed', 'Tue', 'Mon']

# Create the heatmap figure
fig = go.Figure()

fig.add_trace(go.Heatmap(
    x=noise_per_hour['hour'],
    y=noise_per_hour['wk_day'],
    z=noise_per_hour['laeq'], # NOTE: standardizing the data gives the exact same heatmap so no use in doing this
    colorscale='Reds',  
    hovertemplate='Hour: %{x}<br>Weekday: %{y}<br>Sound level: %{z:.2f} dB(A)',
    hoverlabel=dict(namelength=0),
    ygap=1,  # Add gap between the heatmap cells
    yperiodalignment='middle',  # Align the y-axis labels to the middle of each period
))

# set the axis label
fig.update_layout(
    title={
        'text': 'Average noise in Leuven throughout the day',
        'x': 0.5,  # Set the x position to center the title
        'xanchor': 'center',  # Set the anchor point of the title to center
    },
    xaxis=dict(
        title='Time of day',
        tickvals=[0, 6, 12, 18, 23],  
    ),
    yaxis=dict(
        title='Weekday',
        categoryorder='array',
        categoryarray=weekday_order,
    ),
)

# show the figure
fig.show()

In [17]:
# Select rows where the hour is in the desired range
nightly_noise = noise_per_hour[noise_per_hour['hour'].isin([0, 1, 2, 3, 4, 5, 23])].copy()

# Reset the index of the new dataframe
nightly_noise.reset_index(drop=True, inplace=True)

nightly_noise.loc[nightly_noise['hour'] == 23, 'hour'] = -1

In [52]:
# Create the heatmap figure
fig = go.Figure()

fig.add_trace(go.Heatmap(
    x=nightly_noise['hour'],
    y=nightly_noise['wk_day'],
    z=nightly_noise['laeq'], 
    colorscale='Reds',  
    hovertemplate='Hour: %{x}<br>Weekday: %{y}<br>Sound level: %{z:.2f} dB(A)',
    hoverlabel=dict(namelength=0),
    ygap=1,  
    yperiodalignment='middle', 
))

# set the axis label
fig.update_layout(
    title={
        'text': 'Average noise in Leuven at night: Thursday = party day',
        'x': 0.5,  
        'xanchor': 'center',  
    },
    xaxis=dict(
        title='Time of day',
        categoryorder='array',
        categoryarray=[-1, 0, 1, 2, 3, 4, 5],
        tickmode='array',
        tickvals=[-1, 0, 1, 2, 3, 4, 5],
        ticktext=['23', '0', '1', '2', '3', '4', '5'],
        range=[-1.5, 5.5],
    ),
    yaxis=dict(
        title='Weekday',
        categoryorder='array',
        categoryarray=weekday_order,
    ),
)

# Annotate the box for Thursday at hour 4
fig.update_layout(
    annotations=[
        dict(
            x=4,
            y=weekday_order.index('Thu'),
            text='<b>drunk students returning<br>to their dorm</b>',
            showarrow=False,
            font=dict(color='black', size=8),  # Set font weight to 'bold'
            bgcolor='rgba(255, 255, 0, 0)',  # transparent background
            bordercolor='rgba(255, 255, 0, 0)',  # transparent border
        )
    ]
)

# show the figure
fig.show()

In [53]:
# Create the heatmap figure
fig = go.Figure()

fig.add_trace(go.Heatmap(
    x=noise_per_hour['hour'],
    y=noise_per_hour['wk_day'],
    z=noise_per_hour['lamax'], 
    colorscale='Reds',  
    hovertemplate='Hour: %{x}<br>Weekday: %{y}<br>Sound level: %{z:.2f} dB(A)',
    hoverlabel=dict(namelength=0),
    ygap=1,  
    yperiodalignment='middle',  
))

# set the axis label
fig.update_layout(
    title={
        'text': 'Maximal noise in Leuven throughout the day',
        'x': 0.5, 
        'xanchor': 'center',  
    },
    xaxis=dict(
        title='Time of day',
        tickvals=[0, 6, 12, 18, 23],  
    ),
    yaxis=dict(
        title='Weekday',
        categoryorder='array',
        categoryarray=weekday_order,
    ),
)

# show the figure
fig.show()

In [55]:
# Create the heatmap figure
fig = go.Figure()

fig.add_trace(go.Heatmap(
    x=nightly_noise['hour'],
    y=nightly_noise['wk_day'],
    z=nightly_noise['lamax'], 
    colorscale='Reds',  
    hovertemplate='Hour: %{x}<br>Weekday: %{y}<br>Sound level: %{z:.2f} dB(A)',
    hoverlabel=dict(namelength=0),
    ygap=1,  
    yperiodalignment='middle', 
))

# set the axis label
fig.update_layout(
    title={
        'text': 'Maximal noise in Leuven at night: Thursday = party day',
        'x': 0.5,  
        'xanchor': 'center',  
    },
    xaxis=dict(
        title='Time of day',
        categoryorder='array',
        categoryarray=[-1, 0, 1, 2, 3, 4, 5],
        tickmode='array',
        tickvals=[-1, 0, 1, 2, 3, 4, 5],
        ticktext=['23', '0', '1', '2', '3', '4', '5'],
        range=[-1.5, 5.5],
    ),
    yaxis=dict(
        title='Weekday',
        categoryorder='array',
        categoryarray=weekday_order,
    ),
)

# Annotate the box for Thursday at hour 4
fig.update_layout(
    annotations=[
        dict(
            x=4,
            y=weekday_order.index('Thu'),
            text='<b>drunk students returning<br>to their dorm</b>',
            showarrow=False,
            font=dict(color='black', size=8),  # Set font weight to 'bold'
            bgcolor='rgba(255, 255, 0, 0)',  # transparent background
            bordercolor='rgba(255, 255, 0, 0)',  # transparent border
        )
    ]
)

# show the figure
fig.show()